In [1]:
from snowflake.snowpark import Session

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# create_temp_table warning suppresion
import warnings; warnings.simplefilter('ignore')
import configparser

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-s8d05z2d because the default path (/home/mosaic-ai/.cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


# Code to establish connection and read and dump csv as a snowflake table (handling null values before loading)

In [3]:
config = configparser.ConfigParser()
config.read("snowflake_connection.ini")

['snowflake_connection.ini']

In [4]:
connection_parameters = {
    "user": f'{config["Snowflake"]["user"]}',
    "password": f'{config["Snowflake"]["password"]}',
    #"password": os.getenv('snowflake_password'),
    "account": f'{config["Snowflake"]["account"]}',
    #"account": os.getenv('snowflake_account'),
    "WAREHOUSE": f'{config["Snowflake"]["WAREHOUSE"]}',
    "DATABASE": f'{config["Snowflake"]["DATABASE"]}',
    "SCHEMA": f'{config["Snowflake"]["SCHEMA"]}'
}

In [5]:
def snowflake_connector(conn):
    try:
        session = Session.builder.configs(conn).create()
        print("connection successful!")
    except:
        raise ValueError("error while connecting with db")
    return session

session = snowflake_connector(connection_parameters)

connection successful!


In [6]:
application_train  = session.table("CRA_APPLICATION_TRAIN_DETAILS")
application_test  = session.table("CRA_APPLICATION_TEST_DETAILS")
bureau  = session.table("CRA_BUREAU_DETAILS")
bureau_balance  = session.table("CRA_BUREAU_BALANCE_DETAILS")
credit_card_balance  = session.table("CRA_CREDIT_CARD_BALANCE_DETAILS")
installments_payments  = session.table("CRA_INSTALLMENTS_PAYMENTS_DETAILS")
previous_application  = session.table("CRA_PREVIOUS_APPLICATION_DETAILS")
POS_CASH_balance  = session.table("CRA_POS_CASH_BALANCE_DETAILS")